In [ ]:
import os
import json
import pandas as pd
import seaborn as sns

suffixes = ['B', 'KB', 'MB', 'GB', 'TB', 'PB']


def humansize(nbytes):
    i = 0
    while nbytes >= 1024 and i < len(suffixes)-1:
        nbytes /= 1024.
        i += 1
    f = ('%.2f' % nbytes).rstrip('0').rstrip('.')
    return '%s %s' % (f, suffixes[i])

: 

In [ ]:
# Convert Logs to a dataframe
path = "./logs/"
dir_list = os.listdir(path)

logs = []

for file in dir_list:
    with open("./logs/" + file, 'r') as f:
        for line in f.readlines():
            jsonData = json.loads(line)
            if jsonData["level"] == "info" and jsonData["message"] == "FINISH":
                logs.append(jsonData)
                #print(jsonData)

df = pd.DataFrame(logs)

df["time"] = pd.to_datetime(df["time"], format="%Y-%m-%dT%H:%M:%SZ")


In [ ]:
# Connections per day

connPlot = df["time"].groupby(df["time"].dt.date).count().plot(kind="bar")
connPlot.yaxis.grid(True)


In [ ]:
ax = sns.boxplot(x=df["time"].dt.strftime('%b-%y'), y=df["BYTES"])
ax.set(yscale="log")
ax.set_xticklabels(ax.get_xticklabels(), rotation=45,  
                   horizontalalignment="right")
ax.yaxis.grid(True)


In [ ]:
humansize(df.BYTES.sum())


In [ ]:
minutes = df.DURATION.sum()/60000 

print(minutes,  "minutes")
print(minutes/60, "hours")
print(minutes/60/24, "days")

In [ ]:
import re
from collections import Counter

payloads = []

p = re.compile(r'wp-login.php\?(\w*=\w*)')
for row in df.URL:
    result = p.search(row)
    if result != None:
        payloads.append(result.group(1))

Counter(payloads)

In [ ]:
import re 
import base64

useragentPayloads = []

pattern = re.compile(r'(?<=Base64\/)\w+')

for row in df.USERAGENT:
    result = pattern.search(row)
    if result != None:
        useragentPayloads.append(result.group(0))

for payload in useragentPayloads:
    decodedPayload = base64.b64decode(payload + '==')
    print(decodedPayload)